In [ ]:
function ConnectButton() {
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton, 60000);

In [1]:
path_root = '/content/drive/My Drive/Colab Notebooks/DU/'
path_train = path_root + 'data/Corona_NLP_train.csv'
path_test = path_root + 'data/Corona_NLP_test.csv'
path_models = path_root + 'models/'
path_logs = 'logs/'

In [2]:
!rm -rf '{path_models}' '{path_logs}'
!mkdir '{path_models}' '{path_logs}'

In [3]:
%load_ext tensorboard

### Step 1: Data preparation

In [4]:
!pip install unidecode

     |████████████████████████████████| 245kB 14.2MB/s 


In [5]:
import nltk
import html
import unidecode
from string import ascii_lowercase
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize

nltk.download('stopwords')

def clean_text(df):
    df['x'] = [html.unescape(x) for x in df['x_orig']]
    df['x'] = [re.sub(r'https?://\S+', '', x) for x in df['x']]
    df['x'] = [re.sub(r'[^\w\s]|\d+', '', x) for x in df['x']]
    df['x'] = [re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', x) for x in df['x']]
    df['x'] = [re.sub(r'\s\s+|_|\'', ' ', x) for x in df['x']]
    df['x'] = [x.strip().lower() for x in df['x']]
    df['x'] = [unidecode.unidecode(x) for x in df['x']]

    for c in ascii_lowercase:
        df['x'] = [re.sub(c+'{3,}', c+c, x) for x in df['x']]

    #df['x'] = [regexp_tokenize(x, '\w+') for x in df['x']]
    #df['x'] = [' '.join(w for w in x if not w in stopwords.words('english')) for x in df['x']]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
import re
import pandas as pd

def load_csv(path):
    df = pd.read_csv(path, encoding='latin')
    df = df.drop(columns=['UserName', 'ScreenName', 'Location', 'TweetAt'])
    df = df.rename(columns={'OriginalTweet':'x_orig', 'Sentiment':'y'})

    df['y'] = df['y'].apply(lambda x: re.sub('Extremely ', '', x))

    clean_text(df)

    return df

### Step 2: Data exploration

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

y_count = Counter(df['y'])
plt.figure(figsize=(20, 5))
plt.pie(y_count.values(), labels=[class_trans[x] for x in y_count.keys()], autopct='%1.1f%%')
plt.show()

In [ ]:
from wordcloud import WordCloud

for c in classes:
    x = df[df['y'] == class_trans[c]]['x'].to_string()
    plt.imshow(WordCloud().generate(x))
    plt.show()

### Step 3: Pipeline construction

In [ ]:
!pip install transformers
!pip install optuna

In [14]:
import torch
import numpy as np
import tensorflow as tf
from torch.utils.data import TensorDataset
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [9]:
def run_train(config):
    logging.basicConfig(level=logging.INFO)

    tokenizer = config['tokenizer'].from_pretrained(config['name'])

    if config['token_add_special']:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    train = load_csv(path_train)

    data_train = tokenizer.batch_encode_plus(
        train['x'].tolist(),
        truncation=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=config['token_max_length'],
        add_special_tokens=config['token_add_special'],
        return_tensors='pt'
    )

    encoder = LabelEncoder()
    train['y_encd'] = encoder.fit_transform(train['y'])

    dataset_train = tf.data.Dataset.from_tensor_slices((
        data_train,
        train['y_encd'].tolist()
    ))

    args = TFTrainingArguments(
        output_dir=path_models+config['name'],
        num_train_epochs=config['num_epochs'],
        per_device_train_batch_size=config['batch_size'],
        warmup_steps=config['warmup_steps'],
        weight_decay=config['weight_decay'],
        logging_dir=path_logs+config['name'],
        logging_steps=config['logging_steps'],
        load_best_model_at_end=True,
        metric_for_best_model='accuracy'
    )

    with args.strategy.scope():
        model = tokenizer = config['model'].from_pretrained(
            config['name'],
            num_labels=config['num_labels'])

    def compute_metrics(pred):
        preds, labels = eval_pred
        preds = preds.argmax(axis=-1)
        return accuracy_score(labels, preds)

    trainer = TFTrainer(
        model=model,
        args=args,
        train_dataset=dataset_train,
        compute_metrics=compute_metrics
    )

    trainer.train()

    model.save_pretrained(path_models+config['name'])

In [10]:
def run_test(config):
    logging.basicConfig(level=logging.INFO)

    tokenizer = config['tokenizer'].from_pretrained(config['name'])

    if config['token_add_special']:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    test = load_csv(path_test)

    encoder = LabelEncoder()
    test['y_encd'] = encoder.fit_transform(test['y'])

    if config['model_from_tf']:
        model = config['model'].from_pretrained(
            path_models+config['name'],
            num_labels=config['num_labels'],
            from_tf=True)
    else:
        model = config['model'].from_pretrained(
            path_models+config['name'],
            num_labels=config['num_labels'])

    if config['model_to_cuda']:
        device = torch.device('cuda')
        model.to(device)
        model.eval()

    y_true, y_pred = [], []

    for i, row in test.iterrows():
        if config['tensor_type'] == 'tf':
            inputs = tokenizer(row['x'], return_tensors='tf')
            inputs['labels'] = tf.reshape(tf.constant(1), (-1, 1))
            outputs = model(inputs)
            labels = inputs['labels']
            logits = outputs.logits
        elif config['tensor_type'] == 'pt':
            inputs = tokenizer(row['x'], return_tensors='pt').to(device)
            labels = torch.tensor([row['y_encd']]).unsqueeze(0).to(device)
            outputs = model(**inputs, labels=labels)
            labels = labels.detach().cpu().numpy()
            logits = outputs.logits.detach().cpu().numpy()

        y_true.append(labels)
        y_pred.append(logits)

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    y_pred = [encoder.classes_[np.argmax(y)] for y in y_pred]
    y_true = [encoder.classes_[y] for y in y_true]

    print(classification_report(y_true, y_pred))

### Classifier 1: Logistic regression

In [ ]:
%%time
train = load_csv(path_train)
model = Pipeline([('vectorizer', TfidfVectorizer()),
                  ('clf', LogisticRegression(max_iter=500))])
model.fit(train['x'], train['y'])

CPU times: user 1min 57s, sys: 30.5 s, total: 2min 27s
Wall time: 2min 12s


In [ ]:
%%time
test = load_csv(path_test)
y_pred = model.predict(test['x'])
print(classification_report(test['y'], y_pred))

              precision    recall  f1-score   support

    Negative       0.79      0.76      0.78      1633
     Neutral       0.69      0.58      0.63       619
    Positive       0.77      0.84      0.80      1546

    accuracy                           0.77      3798
   macro avg       0.75      0.73      0.74      3798
weighted avg       0.76      0.77      0.76      3798

CPU times: user 10.2 s, sys: 1.32 s, total: 11.6 s
Wall time: 12.8 s


### Classifier 2: DistilBERT

In [10]:
config = {
    'name': 'distilbert-base-uncased',
    'tokenizer': AutoTokenizer,
    'model': AutoModelForSequenceClassification,
    'batch_size': 16,
    'num_epochs': 5,
    'num_labels': 3,
    'learning_rate': 2e-5,
    'warmup_steps': 500,
    'weight_decay': 0.01,
    'logging_steps': 10,
    'token_max_length': 50,
    'token_add_special': False,
    'model_from_tf': False,
    'model_to_cuda': False,
    'tensor_type': 'tf'
}

In [ ]:
%tensorboard --logdir '{path_logs}'{config['name']}

In [ ]:
%%time
run_train(config)

In [ ]:
%%time
run_test(config)

### Classifier 3: DistilGPT2

In [ ]:
config = {
    'name': 'distilgpt2',
    'tokenizer': AutoTokenizer,
    'model': TFAutoModelForSequenceClassification,
    'batch_size': 1,
    'num_epochs': 5,
    'num_labels': 3,
    'learning_rate': 2e-5,
    'warmup_steps': 500,
    'weight_decay': 0.01,
    'logging_steps': 10,
    'token_max_length': 50,
    'token_add_special': True,
    'model_from_tf': True,
    'model_to_cuda': True,
    'tensor_type': 'pt'
}

In [ ]:
%tensorboard --logdir '{path_logs}'{config['name']}

In [ ]:
%%time
run_train(config)

In [ ]:
%%time
run_test(config)

### temp

In [ ]:
config = {
    'name': 'distilbert-base-uncased',
    'tokenizer': AutoTokenizer,
    'model': AutoModelForSequenceClassification,
    'num_labels': 3,
    'token_add_special': False,
    'model_from_tf': False,
    'model_to_cuda': False,
    'tensor_type': 'tf'
}

def run_train(config):
    tokenizer = config['tokenizer'].from_pretrained(config['name'])

    if config['token_add_special']:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    train, test = load_csv(path_train), load_csv(path_test)

    data_train = tokenizer.batch_encode_plus(
        train['x'].tolist(),
        truncation=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=50,
        add_special_tokens=config['token_add_special'],
        return_tensors='pt'
    )

    data_test = tokenizer.batch_encode_plus(
        test['x'].tolist(),
        truncation=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=50,
        add_special_tokens=config['token_add_special'],
        return_tensors='pt'
    )

    encoder = LabelEncoder()
    train['y_encd'] = encoder.fit_transform(train['y'])
    test['y_encd'] = encoder.fit_transform(test['y'])

    dataset_train = TensorDataset(
        data_train['input_ids'],
        data_train['attention_mask'],
        torch.tensor(train['y_encd'].tolist()))

    dataset_test = TensorDataset(
        data_test['input_ids'],
        data_test['attention_mask'],
        torch.tensor(test['y_encd'].tolist()))

    args = TrainingArguments(
        output_dir=path_models+config['name'],
        evaluation_strategy='steps',
        logging_dir=path_logs+config['name'],
        logging_steps=250,
        fp16=True,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy'
    )

    def model_init(trial):
        return config['model'].from_pretrained(
            config['name'],
            num_labels=config['num_labels'])

    def data_collator(features):
        return {
            'input_ids': torch.stack([f[0] for f in features]),
            'attention_mask': torch.stack([f[1] for f in features]),
            'labels': torch.stack([f[2] for f in features])
        }

    def compute_metrics(out):
        label_ids = out.label_ids
        predictions = out.predictions.argmax(-1)
        return {
            'accuracy': accuracy_score(label_ids, predictions)
        }

    callback_stop = EarlyStoppingCallback(
        early_stopping_patience=2,
        early_stopping_threshold=1e-6)

    trainer = Trainer(
        args=args,
        data_collator=data_collator,
        train_dataset=dataset_train,
        eval_dataset=dataset_test,
        tokenizer=tokenizer,
        model_init=model_init,
        compute_metrics=compute_metrics,
        callbacks=[callback_stop]
    )

    best_run = trainer.hyperparameter_search(
        n_trials=10,
        direction='maximize',
        backend='optuna')
    
    print('Best run:', best_run)

    for n, v in best_run.hyperparameters.items():
        setattr(trainer.args, n, v)

    trainer.train()
    model.save_pretrained(path_models+config['name'])

run_train(config)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning:

The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Step,Training Loss,Validation Loss,Accuracy
250,nan,0.886402,0.598999
